## Install Additional Packages

In [ ]:
!pip install -q transformers accelerate langchain_huggingface
!pip install -q langchain_community
!pip install -q "unstructured[all-docs]"
!pip install -q nltk
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Imports

In [ ]:
from tqdm import tqdm
from langchain.docstore.document import Document as LangchainDocument
from typing import Optional, List, Tuple
from google.colab import drive, runtime
from datetime import datetime
import json
import os
from langchain_community.document_loaders import (
    DirectoryLoader,
    UnstructuredMarkdownLoader,
)
from langchain.docstore.document import Document
from langchain import PromptTemplate, LLMChain
from langchain_huggingface.llms import HuggingFacePipeline
from google.colab import drive, runtime
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer, util
from bert_score import score as bert_score
from typing import Optional, List, Tuple, Any, Union, Pattern, Dict, Callable
from collections import Counter
import nltk
from nltk.corpus import stopwords
import torch
import random
import re
import requests

## Utility Functions

In [ ]:
def write_json_file(output_file_path: str, content: Any) -> bool:
  """
  Writes content to json file at provided output path

  Args:
    output_file_path (str): File path at which file should be generated
    content (Any): The content to be written within the file
  
  Returns:
    bool: indicates if writing json was successful or not
  """

  # Get directory name from output file path parameter value
  output_directory = os.path.dirname(output_file_path)

  # If the directory does not exist, create it
  if output_directory and not os.path.exists(output_directory):
    os.makedirs(output_directory)

  # Try writing content to the file - return True if all successful, otherwise catch the error, log and return False
  try:
    with open(output_file_path, "w") as output_file:
      json.dump(content, output_file, indent=4, ensure_ascii=False)

    print("Successfully saved content to {}".format(output_file_path))

    return True
  except(OSError, IOError) as e:
    print("Error saving content to {}".format(output_file_path))
    return False

In [ ]:
def get_oireachtas_question_answers(start_date: str, end_date: str, limit: int):
  """
  API call to oireachtas service to obtain Dáil Éireann leaders questions between provided start and end date with limit value for response size

  Args:
    start_date (str): start date for filter period
    end_date (str): end date for filter period
    limit (int): max documents to be returned
  """

  # Make request to API with requests package
  response = requests.get("https://api.oireachtas.ie/v1/questions?date_start={}&date_end={}&limit={}&show_answers=True".format(start_date, end_date, limit))

  # Read response into json format
  data = response.json()

  # Return response data
  return data

In [ ]:
def extract_qa_pairs(data: Dict[str, Any]) -> List[Dict[str, Any]]:
  """
  Extract question-answer pairs from the provided data - this is response from Oireachtas API

  Args:
    data (Dict[str, Any]): the data from which the QA pairs are to be generated
  
  Returns:
    List[Dict[str, Any]]: list of extract qa pairs
  """

  # Initialise to empty array
  qa_pairs = []
  
  # Loop over results within data if present
  for item in data.get("results", []):
    # Extract question property
    question = item.get("question", {})

    # Extract question and answer text if available from data
    question_text = question.get("showAs", "").strip()
    answer_text = question.get("answerText", "").strip()

    # If both question and answer extracted successfully, append to list
    if len(question_text) > 1 and len(answer_text) > 1:
      qa_pairs.append({**question, "extracted_question": question_text, "extracted_answer": answer_text})

  # Return full list of extracted question-answer pairs
  return qa_pairs

In [ ]:
def evaluate_generated_text(reformulated_text: str, original_text: str, sentence_transformer_model: Any, cosine_threshold: float = 0.7, bert_f1_threshold: float = 0.6) -> bool:
  """
  Evaluate reformulated LLM question-answers with original text to ensure that summarising has not lost core context

  Args:
    reformulated_text (str): generated DeepSeek output after summarising original text
    original_text (str): original text before re-generation with LLM
    sentence_transformer_model: model used to generate sentence embeddings
    cosine_threshold (float): similarity threshold that needs to be exceeded to be deemed a valid LLM generated output
    bert_f1_threshold (float): F1 threshold that needs to be exceeded to be deemed a valid LLM generated output
  
  Returns:
    bool: indicates if reformulated DeepSeek generated output text meets validation criteria when comparing against original text
  """

  # Generate embeddings for both original and reformulated LLM text output
  embedding_original = sentence_transformer_model.encode(original_text, convert_to_tensor=True)
  embedding_reformulated = sentence_transformer_model.encode(reformulated_text, convert_to_tensor=True)

  # Calculate cosine similairity between embeddings
  cosine_similarity = util.cos_sim(embedding_reformulated, embedding_original)

  # Calculate f1 score between original text and re-formulated text by DeepSeek LLM
  _, __, f1 = bert_score([reformulated_text],[original_text], model_type="bert-base-uncased", verbose=False)

  # If both conditions are satisfied, return True, otherwise False
  return cosine_similarity[0] >= cosine_threshold and f1[0] >= bert_f1_threshold

In [ ]:
# Extract oireachtas API data by calling with request
oireachtas_question_answer_data = get_oireachtas_question_answers("2024-01-01", "2025-06-30", 10000)

In [ ]:
print(len(oireachtas_question_answer_data))

In [ ]:
# Write the api data to a file to keep a copy
write_json_file("/content/oireachtas_question_answer_data.json", oireachtas_question_answer_data)

Successfully saved content to /content/oireachtas_question_answer_data.json


True

In [ ]:
# Extract question-answer pairs from the API data
qa_pairs = extract_qa_pairs(oireachtas_question_answer_data)

In [ ]:
print(len(qa_pairs))

954


In [ ]:
# Print for validation
print(qa_pairs[0]["extracted_question"])

1. Deputy David Cullinane asked the Minister for Health the steps she has taken to ensure that Health Service Executive and National Treatment Purchase Fund spending on insourcing initiatives is transparent and accountable; and if she will make a statement on the matter. [34742/25]


In [ ]:
# Print for validation
print(qa_pairs[0]["extracted_answer"])

An Cathaoirleach Gníomhach (Deputy David Maxwell)  Deputy Clarke is asking this question on behalf of Deputy Cullinane. 
 Deputy Sorca Clarke  I thank the Cathaoirleach Gníomhach for facilitating us. 
 Minister for Health (Deputy Jennifer Carroll MacNeill)  The Government remains committed to increasing capacity in the public system. The commitment is underpinned by unparalleled levels of investment as we move to universal access to health care. However, until the capacity is in place to meet the increased levels of demand, it is necessary in the interim to make use of all available capacity in the public and private systems to ensure that patients have access to the care they need.  The waiting list action plan 2025 includes targeting the delivery of additional capacity in the public and private systems through a co-ordinated approach by the HSE and the National Treatment Purchase Fund, NTPF. NTPF insourcing initiatives are governed by a memorandum of understanding, MOU, between the N

In [ ]:
# The distilled 14B DeepSeek model was used for this task (largest that would fit on A100 GPU even with quantisation)
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"

In [ ]:
# Initialise DeepSeek tokeniser
tokeniser = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
# Initialise model, note quantised version
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-000004.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00001-of-000004.safetensors:   0%|          | 0.00/8.71G [00:00<?, ?B/s]

model-00004-of-000004.safetensors:   0%|          | 0.00/3.49G [00:00<?, ?B/s]

model-00002-of-000004.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
# Create transformers pipeline with max new tokens length of 1024
pipe = pipeline("text-generation", model=model, tokenizer=tokeniser, max_new_tokens=1024, do_sample=False)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# Pass pipeline to langchain wrapper
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# Initialise sentence transformer model
sentence_transformer_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Construct prompt (note task is to remove ambiguous political statements, speculation etc). As this dataset is from politician's leader questions,
# the outputs tend to be vague, so this step is needed to summarise the core point if any, of the TDs question to the leader of the house
prompt = PromptTemplate(input_variables=["question", "answer"],
                              template = """Rephrase the following question and answer to state only concrete, verifiable facts.
            Remove opinions, speculation, and political language.
            Do NOT include personal names or titles such as "Deputy".
            If no specific, factual answer is provided, state "No factual information provided".

            Question: {question}

            Answer: {answer}

            Rephrased question:
            [MODEL TO FILL]

            Rephrased answer:
            [MODEL TO FILL]"""
)

In [ ]:
# I mounted my Google Drive to this and ran this task overnight as inference was relatively slow (saving files at checkpoints to ensure minimisation of data loss)
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Declare save path within google drive
SAVE_PATH = "/content/drive/MyDrive/oireachtas_qa_generation_outputs"

In [ ]:
# Ensure target save directory exists in drive
os.makedirs(SAVE_PATH, exist_ok=True)

In [ ]:
# Interval at which generated outputs should be saved to files in Drive
CHECKPOINT_INTERVAL = 50

In [ ]:
# Initialise variables
final_formatted_items = []
generated_answers = 0
qa_chain = LLMChain(llm=llm, prompt=prompt)

# Loop over extracted question-answer pairs
for i, current_item in enumerate(qa_pairs):
  print("{} / {}".format(i + 1, len(qa_pairs)))
  print("*" * 100)
  print("\n")

  question = current_item["extracted_question"]
  answer = current_item["extracted_answer"]

  # If not a valid QA pair, ignore and continue to next iteration
  if len(question) < 2 or len(answer) < 2:
    print("No question or answer extracted from qa dataset item")
    continue

  # Call DeepSeek model with question-answer pair by invoking langchain pipeline
  response = qa_chain.run({"question": question, "answer": answer})

  # Regex patterns for searching for target text in DeepSeek output
  pattern_answer = r"\*\*Rephrased Answer:\*\*\s*(.+?)(?=(?:\*\*Rephrased Question\*\*|$))"
  pattern_question = r"\*\*Rephrased Question:\*\*\s*(.+?)(?=(?:\*\*Rephrased Answer:\*\*|$))"


  # If target patterns present, search thourhg them, otherwise continue to next iteration
  if "Rephrased Question" in response and "Rephrased Answer" in response:

    # Regex search for answer and question patterns
    question_matches = re.findall(pattern_question, response, flags=re.DOTALL)
    answer_matches = re.findall(pattern_answer, response, flags=re.DOTALL)

    # Extract last instance of target question pattern
    if question_matches:
      last_question = question_matches[-1].strip()

    # Extract last instance of target answer pattern
    if answer_matches:
      last_answer = answer_matches[-1].strip()

    # If both initialised, deemed that question-answer has been extracted for validation
    if last_question and last_answer:

      # Determine if the generated output keeps core context of original question answer pair
      valid_response = evaluate_generated_text(last_question, question, sentence_transformer_model)

      # If it does, increment generated answers and store the item
      if valid_response:
        print("Passed validation, appending...")
        generated_answers += 1
        final_formatted_items.append({**current_item, "formatted_question": last_question, "formatted_answer": last_answer})

        # If checkpoint limit hit, generate json file, write all generated qa pairs to Google Drive and reset variables for next checkpoint run
        if generated_answers % CHECKPOINT_INTERVAL == 0:
          timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
          write_json_file("{}/{}_qa.json".format(SAVE_PATH, timestamp), final_formatted_items)
          final_formatted_items = []

      else:
        print("Failed validation")
    else:
      print("Failure extracting from generated text")
  else:
    print("Rephrased Question or Rephrased Answer not in generated text")


# Final check after all runs (any remaining items)
if len(final_formatted_items) > 0:
  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
  write_json_file("{}/{}_qa.json".format(SAVE_PATH, timestamp), final_formatted_items)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


1 / 954
****************************************************************************************************




The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Passed validation, appending...
2 / 954
****************************************************************************************************




The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Rephrased Question or Rephrased Answer not in generated text
896 / 954
****************************************************************************************************




The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Passed validation, appending...
897 / 954
****************************************************************************************************




The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Failed validation
898 / 954
****************************************************************************************************




In [ ]:
# This was used to ensure that the Colab instance would disconnect immediately to minimise use of Colab credits
print("QA generation completed successfully")
runtime.unassign()